In [58]:
import pandas as pd
from groq import Groq
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap, RunnableLambda
from sqlalchemy import create_engine
import tqdm
import time
import pickle


from dotenv import load_dotenv

load_dotenv()

True

In [59]:
table_description = {
'brand_master'  : 'It contains data related to the bmapping of brand id and the name of the brand ',
'cost_center_hierarchy' : 'It contains data related to cost center id  , The name of the cost center, the mapping between a cost center and functional area and also the parent cost center id which indicates the rollup cost center for this cost center',
'cost_element_hierarchy' : 'It contains data related to cost element id  , The name of the cost element, and also the parent cost element id which indicates the rollup cost element for this cost center',
'functional_area_hierarchy' : 'It contains data related to fuctional_area id  , The name of the functional area, and also the parent functional area id which indicates the rollup functional area for this cost center Each  cost center from cost_center_hierarchy is mapped to a functional area id in this table',
'functional_area_metric_map' : 'It contains mapping of a functional area to Profit & Loss metric like Net Sales , Gross Sales , Expenses etc',
'key_figure_metric_map' : 'It contains mapping of a key figures to Profit & Loss metric like Net Sales , Gross Sales , Expenses etc',
'profit_center_hierarchy' : 'It contains data related to profit center id  , The name of the profit center , and also the parent profit center id which indicates the rollup profit center. Each  profit center also maps to the Brand Id from the brand_master table .',
'sales_data' : 'It contains data related the actual sales which is segregated by key figure, version,year month, profit center  and value .Each key_figure maps to a business metric which can be found using key_figure_metric_map.Each Profit Center maps to a Brand id which can be found using profit_center_hierarchy. The actual Brand name can be found using the mapping from brand_master . Each Version name is an identifier that tells if the version is an actual version or budget. Currency tells about the transaction Currency used for the transaction.',
 'income_expense_reporting' : 'It contains data related the actual sales & expenses which is segregated by key figure, version,year month, profit center  and value .Each key_figure maps to a business metric which can be found using key_figure_metric_map.Each Profit Center maps to a Brand id which can be found using profit_center_hierarchy. The actual Brand name can be found using the mapping from brand_master . Each Version name is an identifier that tells if the version is an actual version or budget. Currency tells about the transaction Currency used for the transaction.'
}

In [61]:
from sqlalchemy import create_engine,text
import os
DB_USER=os.getenv("DB_USER")
DB_PASSWORD=os.getenv("DB_PASSWORD")
DB_HOST=os.getenv("DB_HOST")
DB_NAME=os.getenv("DBBASE")
DB_PORT=5432

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?sslmode=require")

with engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    for row in result:
        print(row)


('PostgreSQL 17.5 (1b53132) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit',)


In [62]:
import pandas as pd
from sqlalchemy import create_engine, text

def read_sql(table, engine):
    conn = None
    try:
        # Open a connection
        conn = engine.connect()

        # Query to get shuffled rows and limit to 5
        query = f"SELECT * FROM {table} ORDER BY RANDOM() LIMIT 5;"

        # Execute and load into DataFrame
        df_sample = pd.read_sql(text(query), con=conn)
        return df_sample

    except Exception as e:
        print(f"Error while reading from {table}: {e}")
        return None

    finally:
        # Close connection if it was opened
        if conn is not None:
            conn.close()


In [63]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None
)



In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", """
You are an intelligent data annotator. Please annotate data as mentioned by human and give output without any verbose and without any additional explantion.
You will be given sql table description and sample columns from the sql table. The description that you generate will be given as input to text to sql automated system.
Output of project depends on how you generate description. Make sure your description has all possible nuances.

"""),

    ("human", '''

- Based on the column data, please generate description of entire table along with description for each column and sample values(1 or 2) for each column.
- While generating column descriptions, please look at sql table description given to you and try to include them in column description. 
- DONT write generic description . Just write description based on what you see in columns.

      
Context regarding the tables:
These tables are standard tables provided by  SAP S4 HANA.
In SAP S/4HANA, the Cost Element Hierarchy defines the types of expenses and revenues (e.g., salaries, rent, advertising, sales income) and links financial accounts with controlling objects. Cost Centers represent organizational units such as departments or functions where costs are incurred, and each cost center is assigned to a Functional Area that groups similar activities (like production, sales, or administration) for reporting in Profit & Loss statements. Profit Centers represent business units responsible for profitability and are often associated with brands, divisions, or regions. Sales performance is captured in the Sales Data table, which records key figures (sales, margin), versions (plan, actual, forecast), time periods, and the related profit center. The Income and Expense Reporting table consolidates financial performance by linking cost elements, cost centers, functional areas, and profit centers, providing detailed insights into revenues, costs, and profitability across dimensions.

Relationships:

Cost Elements → Cost Centers: Each cost element is posted to a cost center to track where costs occur.
Cost Centers → Functional Areas: Each cost center belongs to one functional area for P&L grouping.
Profit Centers can aggregate multiple cost centers and capture sales and expense values.
Sales Data → Profit Centers: Sales metrics are assigned to profit centers for profitability analysis.
Income & Expense Reporting brings all together, linking Functional Area, Profit Center, Cost Center, and Cost Element with values, forming the central reporting layer.
    

Please create a JSON structure that includes:

1. Each column name.
2. A few sample values for each column (up to 3 values).

The JSON should be simple, valid, and formatted like this:

{{
  "columns": {{
    "column1": ["sample_value1", "sample_value2", "sample_value3"],
    "column2": ["sample_value1", "sample_value2", "sample_value3"]
  }}
}}

Important constraints:
- Do NOT include any reasoning, explanation, or <think> blocks.
- Do NOT include markdown, code fences, or extra text.
- The output MUST be valid JSON parsable by standard JSON parsers.     
SQL table name :
{sql_tbl_nm}
     
SQL table description:
{description}

Sample rows from the table:
{data_sample}  
     
     ''')
])

# Fix the RunnableMap implementation
chain = (
    RunnableMap({
        "description": lambda x: x["description"],
        "data_sample": lambda x: x["data_sample"],
        "sql_tbl_nm": lambda x: x["sql_tbl_nm"],
    })
    | template
    | model
    | StrOutputParser()
)

In [67]:
import json
import re
import time
import tqdm
from sqlalchemy import create_engine

# Initialize DB engine
engine = create_engine(
    f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?sslmode=require"
)

# Safe parser function
def parse_llm_json(text):
    """
    Cleans up LLM output by removing <think> blocks and parsing JSON.
    Returns a Python object (dict/list) or None if parsing fails.
    """
    # Remove <think>...</think>
    cleaned = re.sub(r"<think>.*?</think>\n*", "", text, flags=re.DOTALL).strip()
    match = re.search(r"\{.*\}", cleaned, flags=re.DOTALL)
    if match:
        return json.loads(match.group(0))
    return None

    
# Dictionary to store final results
kb_final = {}

for table_name, table_desc in tqdm.tqdm(table_description.items()):
    # 1️⃣ Read sample data from the DB
    df_sample = read_sql(table_name, engine)

    # 2️⃣ Convert DataFrame to JSON-friendly structure
    data_sample_json = df_sample.to_dict(orient="records")

    # 3️⃣ Invoke LLM
    raw_response = chain.invoke({
        "description": table_desc,
        "data_sample": json.dumps(data_sample_json),
        "sql_tbl_nm" : table_name
    }).replace('```', '')  # remove markdown code fences if any

    print(raw_response)
    print('====================================================')

    # 4️⃣ Parse LLM response safely into Python dict/list
    kb_final[table_name] = parse_llm_json(raw_response)
    break
    # 5️⃣ Optional: small delay between requests
    time.sleep(5)

# kb_final now contains proper Python objects keyed by table name


  0%|          | 0/9 [00:03<?, ?it/s]

<think>
Alright, I need to create a JSON structure based on the provided SQL table information. The table is named "brand_master" and contains data about brand mappings, specifically brand_id and brand_name. 

First, I'll start by setting the table name in the JSON. Then, I'll move on to the columns. There are two columns: "brand_id" and "brand_name". 

For each column, I need to provide a few sample values. Looking at the sample rows given, I can extract values for each column. For "brand_id", the samples are "B002", "B003", and "B001". For "brand_name", the samples are "Beta Brand", "Gamma Brand", and "Alpha Brand".

I'll structure the JSON accordingly, ensuring it's valid and follows the specified format. I'll make sure not to include any extra text or markdown, keeping it clean and parsable.
</think>

{
  "table_name": "brand_master",
  "columns": {
    "brand_id": ["B002", "B003", "B001"],
    "brand_name": ["Beta Brand", "Gamma Brand", "Alpha Brand"]
  }
}


In [68]:
kb_final

{'brand_master': {'table_name': 'brand_master',
  'columns': {'brand_id': ['B002', 'B003', 'B001'],
   'brand_name': ['Beta Brand', 'Gamma Brand', 'Alpha Brand']}}}

In [30]:
eval(test)

TypeError: eval() arg 1 must be a string, bytes or code object